In [1]:
import pandas as pd
import numpy as np

# Models
import keras
from keras.models import Sequential
from keras.layers import Dense

#Tuning and Cross Validation
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#### Importing Data

In [2]:
x_train = pd.read_csv('../../Data Files/Training Data/x_train.csv')
x_test = pd.read_csv('../../Data Files/Training Data/x_test.csv')
y_train = pd.read_csv('../../Data Files/Training Data/y_train.csv')
y_test = pd.read_csv('../../Data Files/Training Data/y_test.csv')

#### Defining Neural Network Architecture

In [3]:
def build_nn_model(num_neurons_1, num_neurons_2, activation_fn, optimizer_fn):
    nn = Sequential()
    nn.add(Dense(num_neurons_1, input_shape=[len(x_train.columns),], activation=activation_fn))
    nn.add(Dense(num_neurons_2, input_shape=[len(x_train.columns),], activation=activation_fn))
    nn.add(Dense(1, activation='linear'))
    nn.compile(loss='mean_squared_error', optimizer=optimizer_fn, metrics=['mean_absolute_error']) 
    return nn

#### Training Neural Network

In [4]:
nn = build_nn_model(30, 30, 'relu', 'adam')
nn_history = nn.fit(x_train, y_train, batch_size=5, epochs=5, verbose=1)
y_pred_nn = nn.predict(x_test)

Epoch 1/5
2496/2496 [==============================] - 6s 2ms/step - loss: 4.3448 - mean_absolute_error: 1.4877
Epoch 2/5
2496/2496 [==============================] - 5s 2ms/step - loss: 2.3938 - mean_absolute_error: 1.1826
Epoch 3/5
2496/2496 [==============================] - 5s 2ms/step - loss: 2.2621 - mean_absolute_error: 1.1381
Epoch 4/5
2496/2496 [==============================] - 5s 2ms/step - loss: 2.2145 - mean_absolute_error: 1.1257
Epoch 5/5
98/98 [==============================] - 0s 1ms/step


#### Tuning for Hyperparameters with Cross Validation

In [5]:
model = KerasRegressor(build_fn=build_nn_model)

param_grid = {'num_neurons_1': [30, 50],
              'num_neurons_2': [10, 30, 50],
              'activation_fn': ['tanh', 'softplus', 'relu'],
              'optimizer_fn': ['adam', 'sgd'],
              'batch_size': [16, 64],
              'epochs': [10]}

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=skf, verbose=3)
grid_search.fit(x_train, y_train, verbose=0)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


c:\Users\Guo Xiong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


260/260 [==============================] - 0s 2ms/step
[CV 1/3] END activation_fn=tanh, batch_size=16, epochs=10, num_neurons_1=30, num_neurons_2=10, optimizer_fn=adam;, score=-1.116 total time=   9.4s
260/260 [==============================] - 0s 2ms/step
[CV 2/3] END activation_fn=tanh, batch_size=16, epochs=10, num_neurons_1=30, num_neurons_2=10, optimizer_fn=adam;, score=-1.116 total time=   9.4s
260/260 [==============================] - 0s 1ms/step
[CV 3/3] END activation_fn=tanh, batch_size=16, epochs=10, num_neurons_1=30, num_neurons_2=10, optimizer_fn=adam;, score=-1.128 total time=   9.5s
260/260 [==============================] - 0s 1ms/step
[CV 1/3] END activation_fn=tanh, batch_size=16, epochs=10, num_neurons_1=30, num_neurons_2=10, optimizer_fn=sgd;, score=-1.099 total time=   8.4s
260/260 [==============================] - 0s 1ms/step
[CV 2/3] END activation_fn=tanh, batch_size=16, epochs=10, num_neurons_1=30, num_neurons_2=10, optimizer_fn=sgd;, score=-1.119 total time=

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=1, shuffle=True),
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000001E4866FBE80>,
             param_grid={'activation_fn': ['tanh', 'softplus', 'relu'],
                         'batch_size': [16, 64], 'epochs': [10],
                         'num_neurons_1': [30, 50],
                         'num_neurons_2': [10, 30, 50],
                         'optimizer_fn': ['adam', 'sgd']},
             scoring='neg_mean_absolute_error', verbose=3)

In [6]:
optimal_nn = grid_search.best_estimator_

y_pred = optimal_nn.predict(x_test)
mae_after_tuning = mean_absolute_error(y_test, y_pred)
print("mean_absolute_error = {:.3}".format(mae_after_tuning)) 
print(grid_search.best_params_)

195/195 [==============================] - 0s 2ms/step
mean_absolute_error = 1.12
{'activation_fn': 'tanh', 'batch_size': 16, 'epochs': 10, 'num_neurons_1': 50, 'num_neurons_2': 50, 'optimizer_fn': 'adam'}


#### Generating Predictions

In [19]:
best_params = grid_search.best_params_
tuned_nn = build_nn_model(best_params['num_neurons_1'], best_params['num_neurons_2'], best_params['activation_fn'], best_params['optimizer_fn'])
nn_history = tuned_nn.fit(x_train, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=1)
y_pred_nn = tuned_nn.predict(x_test)

Epoch 1/10
780/780 [==============================] - 3s 3ms/step - loss: 3.2643 - mean_absolute_error: 1.3275
Epoch 2/10
780/780 [==============================] - 2s 3ms/step - loss: 2.2213 - mean_absolute_error: 1.1229
Epoch 3/10
780/780 [==============================] - 2s 2ms/step - loss: 2.1937 - mean_absolute_error: 1.1175
Epoch 4/10
780/780 [==============================] - 2s 3ms/step - loss: 2.1743 - mean_absolute_error: 1.1136
Epoch 5/10
780/780 [==============================] - 2s 2ms/step - loss: 2.1734 - mean_absolute_error: 1.1114
Epoch 6/10
780/780 [==============================] - 2s 2ms/step - loss: 2.1694 - mean_absolute_error: 1.1116
Epoch 7/10
780/780 [==============================] - 2s 2ms/step - loss: 2.1651 - mean_absolute_error: 1.1099
Epoch 8/10
780/780 [==============================] - 2s 2ms/step - loss: 2.1537 - mean_absolute_error: 1.1061
Epoch 9/10
780/780 [==============================] - 2s 2ms/step - loss: 2.1463 - mean_absolute_error: 1.1054
E

#### Saving Model File and Predictions

In [13]:
save_path = '../../Data Files/'
tuned_nn.save(save_path + 'Model Files/' + 'nn.h5')
np.savetxt(save_path + 'Predictions/' + 'neuralnetwork_output.csv', y_pred_nn, delimiter=",")